## library

In [5]:
import os.path as osp
import cv2

In [2]:
# 学習、検証の画像データとアノテーションデータへのファイルパスリストを作成する

def make_datapath_list(rootpath):
    """
    データへのパスを格納したリストを作成する
    
    Params:
    -------------
    rootpath : str
        データフォルダへのパス
    
    Returns:
    -------------
    ret : train_img_list, train_anno_list, val_img_list, val_anno_list
        データのへのパスを格納したリスト      
    """
    
    # 画像ファイルとアノテーションファイルへのパスのテンプレートを作成
    imgpath_template = ops.join(rootpath, 'JPEGImages', '%s.jpg')　　　# テンプレートの書き方がわからない
    annopath_tempate = ops.join(rootpath, 'Annotations', '%s.xml')
    
    # 訓練と検証、それぞれのファイルのID（ファイル名）を取得する
    train_id_names  = ops.join(rootpath+'ImageSets/Main/train.txt')
    val_id_names = ops.join(rootpaht+'ImageSets/Main/val.txt')
    
    # 訓練データの画像ファイルとアノテーションファイルへのパスのリストを作成
    trian_img_list = list()
    train_anno_list = list()
    
    for line in open(train_id_names):
        file_id = line.strip()  # 空白スペースと改行を除去
        img_path = (imgpath_template % file_id)      # 画像のパス  なぜタプルで囲む必要があるのか
        anno_path = (annopath_tempate % file_id)   # アノテーションのパス
        train_img_names.append(img_path)               # リストに追加
        train_anno_names.append(anno_path)           # リストに追加
    
    # 検証データの画像ファイルとアノテーションファイルへのパスのリストを作成
    val_img_list = list()
    val_anno_list = list()
    
    for line in open(val_id_names):
        file_id = line.strip()  # 空白スペースと改行を除去
        img_path = (imgpath_template % file_id)      # 画像のパス
        anno_path = (annopath_tempate % file_id)   # アノテーションのパス
        val_img_names.append(img_path)               # リストに追加
        val_anno_names.append(anno_path)           # リストに追加
    
    return  trian_img_list, train_anno_list, val_img_list, val_anno_list

In [8]:
# template test
test_path = osp.join('JPEGImages', '%s.jpg')
file_id  = '000000'
img_path = (test_path % file_id)  
print(img_path)

JPEGImages/000000.jpg


In [ ]:
# ファイルパスのリストを作成
rootpath = './data/VOCdevkit/VOC2012/'
trian_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

# 動作確認
print(train_img_list[0])

In [10]:
# listにlist足したらどうなるのか
ret = []
a = ['a', 'i', 'u', 'e' ,'o']
b = ['ka', 'ki', 'ku', 'ke' ,'ko']
ret += a 
print(ret)
ret += b
print(ret)

['a', 'i', 'u', 'e', 'o']
['a', 'i', 'u', 'e', 'o', 'ka', 'ki', 'ku', 'ke', 'ko']


In [13]:
# listをlistで囲ってlistに足したらどうなるのか
ret = []
a = ['a', 'i', 'u', 'e' ,'o']
b = ['ka', 'ki', 'ku', 'ke' ,'ko']
ret += [a] 
print(ret)
ret += [b]
print(ret)

# これをnumpyにしたら
import numpy as np
np.array(ret)

[['a', 'i', 'u', 'e', 'o']]
[['a', 'i', 'u', 'e', 'o'], ['ka', 'ki', 'ku', 'ke', 'ko']]


array([['a', 'i', 'u', 'e', 'o'],
       ['ka', 'ki', 'ku', 'ke', 'ko']], dtype='<U2')

In [ ]:
# XML形式のアノテーションデータをpythonのリスト形式に変換するクラス

class Anno_xml2list(object):
    """
    一枚の画像に対する「ｘml形式のアノテーションデータ」を画像サイズに規格化してからリスト形式に変換する
    
    Attributeｓ
    -------------
    classes : list
        VOCのクラス名を格納したリスト
    """
    
    def __init__(self, classes):
        self.classes = classes
        
    def __call__(self, xml_path, width, hetig):
        """
        一枚の画像に対する「ｘml形式のアノテーションデータ」を画像サイズに規格化してからリスト形式に変換する
        
        Params
        -------------
        xml_path : str
            xmlファイルへのパス
        width : int
            対象画像の幅
        height : int
            対象画像の高さ
            
        Returns
        -------------
        ret : [[xmin, ymin, xmax, ymax, label_ind], ...]
            物体のアノテーションデータを格納したリスト
            画像内に存在する物体数分の要素を持つ
            一つの画像につき５つの値が与えられている
        """
        
        # 画像内の全ての物体のアノテーションをこのリストに格納
        ret = []
        
        # xmlファイルの読み込み
        xml = ET.parse(xml_path).getroot()
        
        # 画像内にある物体(object)の数だけループする
        for obj in xml.iter('object'):
            # アノテーションで検知がdifficultに設定されているものを除外
            difficult = int(obj.find('difficult').text)   # この書きかた分かってない　find()
            if difficult == 1:
                continue
            
            # 一つの物体に対するアノテーションを格納するリスト
            bndbox = []
            name = obj.find('name').text.lower().strip()  # 物体名
            bbox = obj.find('bbox')                                      # バウンディングボックスの情報
            
            # アノテーションの xmin, ymin, xmax, ymax, label_ind を取得し、0-1に正規化
            pts = ['xmin', 'ymin', 'xmax', 'ymax']
            
            for pt in (pts):
                # なぜタプルで囲ったのか
                # 原点が(1,1)なので(0,0)に修正
                cur_pixel = int(bbox.find(pt).text) - 1
                
                # 幅、高さで規格化
                if pt == 'xmin' or pt == 'xmax':       # x軸の方向のときは幅で割り算
                    cur_pixel  /= width
                else:                                                    # y軸のときは高さで割り算
                    cur_pixel /= height
                
                bndbox.append(cur_pixel)
                
            # アノテーションのクラス名のindexを取得して追加
            label_idx = self.classes.index(name)
            bndbox.append(lable_idx)
            
            # res に[[xmin, ymin, xmax, ymax, label_ind]]を追加する
            res += [bndbox]
            
        return np.array(ret)  # [[xmin, ymin, xmax, ymax, label_ind], ...]

In [ ]:
# Anno_xml2listの動作確認
voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat', 
               'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 
               'diningtable', 'dog', 'horse', 'motorbike', 
               'person', 'pottedplant', 'sheep', 'sofa', 
              'train', 'tvmonitor']

# ここで__init__
transform_anno = Anno_xml2list(voc_classes)

# 画像の読み込み　OpenCVを使用
ind = 1
image_file_path = val_img_list[ind]
img = cv2.imread(image_file_path)   # 高さ、幅、色RGB
height, width, channels = img.shape   # 画像サイズの取得

# アノテーションをリストで表示
# ここで__call__
transform_anno(val_anno_list[ind], width, height)